# Memory and Stream Objects

This notebook is a companion to the _Exploring a Design_ notebook in the getting started guide. Whereas that notebook looked at the dictionaries that are available before a bitstream is downloaded, here we look in greater detail at how memories and streams can be interacted with once the overlay is loaded.

Again we are going to use the `kernel_opt.xclbin` file as it provides a wide range of features that can be inspected.

In [1]:
import pynq

ol = pynq.Overlay('kernel_opt.xclbin')

## Memory Status

As well as being used as a target for memory allocation, the memory attributes can also get some basic statistics of memory usage in the system as a whole.

In [2]:
bank = ol.bank1

bank.mem_used

0

Allocating a 4 MB array shows a corresponding jump in the used memory

In [3]:
buf = pynq.allocate((1024,1024), 'u4', target=bank)

bank.mem_used

4194304

And deleting the array frees the underlying hardware resources

In [4]:
%xdel buf

bank.mem_used

0

The reason for using `%xdel` rather than the regular Python `del` is that Jupyter can keep references to variables around even after they've been deleted which prevents the object from being freed. If we repeat the experiment but using `del` and printing the contents of the buffer we can see that memory is not freed.

In [5]:
buf = pynq.allocate((1024,1024), 'u4', target=bank)
buf

PynqBuffer([[0, 0, 0, ..., 0, 0, 0],
            [0, 0, 0, ..., 0, 0, 0],
            [0, 0, 0, ..., 0, 0, 0],
            ...,
            [0, 0, 0, ..., 0, 0, 0],
            [0, 0, 0, ..., 0, 0, 0],
            [0, 0, 0, ..., 0, 0, 0]], dtype=uint32)

In [6]:
del buf
bank.mem_used

4194304

We can reference this instance using `_*` notataion that Jupyter provides to access old cell results. If you've rerun cells in this notebook you will need to change the code below to refer to the cell printing the buffer.

In [7]:
buf = _5

Running `%xdel` on our reclaimed reference will strip all references out of IPython. This is important to note as we cannot reprogram the bitstream if there are buffers still allocated.

In [8]:
%xdel buf
bank.mem_used

0

## Stream Information

The attributes representing stream objects provide a straightforward way to get information about the stream. The `source` and `sink` attributes of the stream object provide a textual representation in `'{ip}.{port}'` format

In [9]:
stream = ol.dc_0

stream.source

'krnl_stream_vadd_1.out_r'

The the IP driver has been instantiated then the `source_ip` or `sink_ip` attributes will provide direct access to the driver for the IP the endpoint is attached to.

In [10]:
vadd = ol.krnl_stream_vadd_1
stream.source_ip

## Cleaning Up

To release the card for others we should `free` the overlay when we are finished with it

In [11]:
ol.free()

Copyright (C) 2020 Xilix, Inc